<a href="https://colab.research.google.com/github/sdmarshall27/Mancala-1.0/blob/master/Preliminaries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Preliminaries For Deep Learning

1. Imports




In [1]:
import collections
import hashlib
import math
import os
import random
import re
import shutil
import sys
import tarfile
import time
import zipfile
from collections import defaultdict
import pandas as pd
import requests
from IPython import display
from matplotlib import pyplot as plt
d2l = sys.modules[__name__]

In [2]:
!sudo ln -sfT /usr/local/cuda/cuda-11.0/ /usr/local/cuda
!pip install mxnet-cu110

import mxnet
mxnet.__version__

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 327.3 MB 5.7 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


'1.9.1'

In [ ]:
pip install mxnet==1.7.0.post1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55.0 MB 121 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


To start, we import the np (numpy) and npx (numpy_extension) modules from MXNet. Here, the np module includes functions supported by NumPy, while the npx module contains a set of extensions developed to empower deep learning within a NumPy-like environment. When using tensors, we almost always invoke the set_np function: this is for compatibility of tensor processing by other components of MXNet.

In [ ]:
from mxnet import autograd, context, gluon, image, init, np, npx
from mxnet.gluon import nn, rnn

In [ ]:
npx.set_np()

2. Tensors

A tensor represents a (possibly multi-dimensional) array of numerical values. With one axis, a tensor corresponds (in math) to a vector. With two axes, a tensor corresponds to a matrix. Tensors with more than two axes do not have special mathematical names.

In [ ]:
x = np.arange(12)
x

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11.])

In [ ]:
x.shape #Tensor's shape (the length along each axis.)

(12,)

In [ ]:
x.size #Tensor's size

12

In [ ]:
X = x.reshape(3,4)
X

array([[ 0.,  1.,  2.,  3.],
       [ 4.,  5.,  6.,  7.],
       [ 8.,  9., 10., 11.]])

In [ ]:
np.zeros((2,3,4))

array([[[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]]])

In [ ]:
np.ones((3,3,4))

array([[[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]],

       [[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]],

       [[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]]])

The following snippet creates a tensor with shape (3, 4). Each of its elements is randomly sampled from a standard Gaussian (normal) distribution with a mean of 0 and a standard deviation of 1.

In [ ]:
np.random.normal(0,1,size = (3,4))

array([[ 2.2122064 ,  1.1630787 ,  0.7740038 ,  0.4838046 ],
       [ 1.0434403 ,  0.29956347,  1.1839255 ,  0.15302546],
       [ 1.8917114 , -1.1688148 , -1.2347414 ,  1.5580711 ]])

3. Operations on Tensors

In [ ]:
x = np.array([1,2,3,4])
y = np.array([5,6,7,8])
x+y, x-y, x*y, x/y, x ** y 

(array([ 6.,  8., 10., 12.]),
 array([-4., -4., -4., -4.]),
 array([ 5., 12., 21., 32.]),
 array([0.2       , 0.33333334, 0.42857143, 0.5       ]),
 array([1.0000e+00, 6.4000e+01, 2.1870e+03, 6.5536e+04]))

We can also concatenate multiple tensors together, stacking them end-to-end to form a larger ten- sor. We just need to provide a list of tensors and tell the system along which axis to concatenate. The example below shows what happens when we concatenate two matrices along rows (axis 0, the first element of the shape) vs. columns (axis 1, the second element of the shape). We can see that the first output tensorʼs axis-0 length (6) is the sum of the two input tensorsʼ axis-0 lengths (3 + 3); while the second output tensorʼs axis-1 length (8) is the sum of the two input tensorsʼ axis-1 lengths (4 + 4).

In [ ]:
X = np.arange(12).reshape(3,4)
Y = np.array([[2,1,4,3],[1,2,3,4],[4,3,2,1]])
np.concatenate([X,Y],axis = 0) , np.concatenate([X,Y],axis = 1)

(array([[ 0.,  1.,  2.,  3.],
        [ 4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11.],
        [ 2.,  1.,  4.,  3.],
        [ 1.,  2.,  3.,  4.],
        [ 4.,  3.,  2.,  1.]]),
 array([[ 0.,  1.,  2.,  3.,  2.,  1.,  4.,  3.],
        [ 4.,  5.,  6.,  7.,  1.,  2.,  3.,  4.],
        [ 8.,  9., 10., 11.,  4.,  3.,  2.,  1.]]))

In [ ]:
#Binary Tensors
X == Y

array([[False,  True, False,  True],
       [False, False, False, False],
       [False, False, False, False]])

In [ ]:
#Sum of all elements in the tensor
X.sum()

array(66.)

4. Broadcasting Mechanism \\
 Under certain conditions, even when shapes differ, we can still perform elementwise op- erations by invoking the broadcasting mechanism. This mechanism works in the following way: First, expand one or both arrays by copying elements appropriately so that after this transforma- tion, the two tensors have the same shape. Second, carry out the elementwise operations on the resulting arrays.
In most cases, we broadcast along an axis where an array initially only has length 1, such as in the following example:

In [ ]:
a = np.arange(3).reshape(3,1) # 3x1 matrix
b = np.arange(2).reshape(1,2) # 1x2 matrix
a,  b 

(array([[0.],
        [1.],
        [2.]]), array([[0., 1.]]))

In [ ]:
a + b

array([[0., 1.],
       [1., 2.],
       [2., 3.]])

5. Indexing and Slicing

In [ ]:
X[-1], X[1:3]

(array([ 8.,  9., 10., 11.]), array([[ 4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11.]]))

In [ ]:
X[1,2] = 9
X

array([[ 0.,  1.,  2.,  3.],
       [ 4.,  5.,  9.,  7.],
       [ 8.,  9., 10., 11.]])

In [ ]:
X[0:2, :] = 12
X

array([[12., 12., 12., 12.],
       [12., 12., 12., 12.],
       [ 8.,  9., 10., 11.]])

5. Saving Memory

In [ ]:
before = id(Y)
Y = Y + X
id(Y) == before

False

In [ ]:
B. Data Preprocessing

In [ ]:
import os